## Código para traduzir o dataset de espanhol para português

In [12]:
from huggingface_hub import login
from google.colab import userdata
import transformers
import pandas as pd
import torch
import json
import gc
import os
from datasets import Dataset
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def traduzir(pasta, batch_size=5):

    # Definir o token!!!
    hf_token = userdata.get('HF_TOKEN')
    login(hf_token)

    model_id = "google/gemma-3-4b-it"

    # Ajeitar para o path correto!!!
    dataset_path = f"/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX/{pasta}"

    # Carregar dados diretamente do JSON
    with open(f"{dataset_path}/{pasta}_data.json", "r", encoding="utf-8") as f:
        dados = json.load(f)

    print(f"📊 Carregados {len(dados)} exemplos da pasta {pasta}")

    # Configuração do modelo
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

    pipeline = transformers.pipeline(
        task="text-generation",
        model=model_id,
        tokenizer=tokenizer,
        dtype=torch.bfloat16,
        device_map="auto",
    )

    # Path de saída
    json_output_path = f"{dataset_path}/{pasta}_data_translated.json"

    anotacoes = []

    # Processar os dados diretamente
    for i, objeto in enumerate(dados):
        print(f"🔄 Processando {i+1}/{len(dados)} - {objeto['MEME-ID']}")

        resultado = {
            "MEME-ID": objeto["MEME-ID"],
            "text": objeto["text"],
            "description": objeto["description"]
        }

        # Traduzir campos individualmente
        for campo in ["text", "description"]:
            if objeto[campo] and objeto[campo].strip():
                try:
                    prompt = f"Somente traduza '{objeto[campo]}' do espanhol para o português do Brasil, mantendo o mesmo nível de formalidade e estilo original."

                    messages = [{"role": "user", "content": prompt}]

                    outputs = pipeline(
                        messages,
                        max_new_tokens=256,
                        do_sample=True,
                        temperature=0.3,
                    )

                    # Extrair a tradução corretamente
                    generated_text = outputs[0]["generated_text"][-1]["content"]
                    resultado[campo] = generated_text
                    print(f"  ✅ {campo} traduzido")

                except Exception as e:
                    print(f"  ❌ Erro ao traduzir {campo}: {e}")
                    resultado[campo] = objeto[campo] 

        anotacoes.append(resultado)

        # Salvar progresso a cada 5 itens
        if (i + 1) % 5 == 0:
            with open(json_output_path, "w", encoding="utf-8") as f:
                json.dump(anotacoes, f, ensure_ascii=False, indent=4)
            print(f"-> Progresso salvo: {i+1}/{len(dados)}")

            # Limpar cache da GPU
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    # Salvar resultado final
    with open(json_output_path, "w", encoding="utf-8") as f:
        json.dump(anotacoes, f, ensure_ascii=False, indent=4)

    # Limpeza
    del pipeline
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

    print(f"✅ Tradução concluída! {len(anotacoes)} itens processados.")

In [ ]:
for pasta in ["test", "validation", "train"]:
    traduzir(pasta)
    print(f"Pasta {pasta} está pronta!")